## 1.Export

###  get airport_T_df

In [1]:
import pandas as pd
import sqlalchemy
sys.path.append('..\keys')
import key

schema="gans_db"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password=key.SQL_PASSWORD # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
airport_T_df = pd.read_sql_table('airport',con=con)

In [2]:
airport_T_df.icao[0:2]

0    AZ-0001
1       EDDH
Name: icao, dtype: object

### Connect to API and collect Data

In [3]:
RERUN_TIME_IN_HOURS = 24
FLIGHT_FORCAST_TIMEWINDOW = 12

In [4]:
import math
from pytz import timezone
from datetime import datetime, date, timedelta
import requests 
import json

In [5]:
def connect_flight_api(airport_df):
    # Prepare URL inputs
    today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    rerun_window = math.ceil(RERUN_TIME_IN_HOURS/24) #roundup to bigger integer
    next_run = (today + timedelta(days=rerun_window))
    times = [["00:00","11:59"],["12:00","23:59"]]
    
    flight_df_l = []
    #Loop over all airports
    for icao in airport_df.icao[1:2]:
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{next_run}T{time[0]}/{next_run}T{time[1]}"
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
            headers = {
				"X-RapidAPI-Key": f"{key.Flight_API_key}",
				"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
			}
            response = requests.request("GET", url, headers=headers, params=querystring)
            response.raise_for_status()
            flights_12h_json = response.json()
            for flight in flights_12h_json["arrivals"]:
                flight_dict = {}
                flight_dict['arrival_icao']         = icao
                flight_dict["arrival_time_local"]   = flight["arrival"].get('scheduledTimeLocal', pd.NaT)
                flight_dict["arrival_terminal"]     = flight['arrival'].get('terminal', "unknown")
                flight_dict["departure_city"]       =  flight["departure"]["airport"].get("name", "unknown")
                flight_dict["departure_icao"]       =  flight["departure"]["airport"].get("icao", "unknown")
                flight_dict["departure_time_local"] = flight["departure"].get("scheduledTimeLocal", pd.NaT)
                flight_dict["airline"]              =  flight["airline"].get("name", "unknown")
                flight_dict["flight_number"]        =  flight.get("number", "unknown")
                flight_dict["data_retrieved_on"]    =  datetime.now().astimezone(timezone('Europe/Berlin')).date()
                flight_df_l.append(flight_dict)
    return flight_df_l

In [6]:
#airport_id_l = ["EGBB","EPWA"]
flights_df_l = connect_flight_api(airport_T_df)

In [7]:
flight_df = pd.DataFrame(flights_df_l)

In [8]:
len(flight_df)

82

In [9]:
flight_df

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EDDH,2023-01-14 06:10+01:00,1,Saint Martin,TNCM,NaT,Air X Charter,AXY 449P,2023-01-13
1,EDDH,2023-01-14 07:35+01:00,2,Frankfurt-am-Main,EDDF,2023-01-14 06:30+01:00,Lufthansa,LH 2,2023-01-13
2,EDDH,2023-01-14 08:50+01:00,2,Zurich,LSZH,2023-01-14 07:20+01:00,SWISS,LX 1050,2023-01-13
3,EDDH,2023-01-14 08:55+01:00,2,Vienna,LOWW,2023-01-14 07:20+01:00,Austrian,OS 175,2023-01-13
4,EDDH,2023-01-14 09:25+01:00,2,London,EGLL,2023-01-14 06:50+00:00,British Airways,BA 964,2023-01-13
...,...,...,...,...,...,...,...,...,...
77,EDDH,2023-01-14 22:35+01:00,1,Tenerife Island,GCTS,2023-01-14 16:25+00:00,Eurowings,EW 7533,2023-01-13
78,EDDH,2023-01-14 22:35+01:00,2,Munich,EDDM,2023-01-14 21:15+01:00,Lufthansa,LH 2088,2023-01-13
79,EDDH,2023-01-14 22:35+01:00,2,Frankfurt-am-Main,EDDF,2023-01-14 21:30+01:00,Lufthansa,LH 36,2023-01-13
80,EDDH,2023-01-14 22:05+01:00,1,Istanbul,LTFM,2023-01-14 20:35+03:00,Turkish,TK 1667,2023-01-13


In [10]:
flight_df.arrival_terminal.unique()

array(['1', '2'], dtype=object)

## 2.Transform

In [12]:
flight_c_df = flight_df.copy()

### change timestamp type

In [13]:
flight_c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   arrival_icao          82 non-null     object
 1   arrival_time_local    82 non-null     object
 2   arrival_terminal      82 non-null     object
 3   departure_city        82 non-null     object
 4   departure_icao        82 non-null     object
 5   departure_time_local  78 non-null     object
 6   airline               82 non-null     object
 7   flight_number         82 non-null     object
 8   data_retrieved_on     82 non-null     object
dtypes: object(9)
memory usage: 5.9+ KB


In [14]:
flight_c_df.arrival_time_local = pd.to_datetime(flight_c_df.arrival_time_local,utc=True)

In [15]:
flight_c_df.data_retrieved_on = pd.to_datetime(flight_c_df.data_retrieved_on)

In [16]:
flight_c_df.loc[flight_c_df.departure_time_local == pd.NaT]

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on


In [17]:
flight_c_df.departure_time_local = pd.to_datetime(flight_c_df.departure_time_local,utc=True)

In [18]:
flight_c_df.departure_time_local.sample(10)

51   2023-01-14 12:55:00+00:00
56   2023-01-14 16:00:00+00:00
28   2023-01-14 07:45:00+00:00
68   2023-01-14 15:45:00+00:00
65   2023-01-14 18:00:00+00:00
15   2023-01-14 08:10:00+00:00
71   2023-01-14 17:20:00+00:00
44   2023-01-14 12:35:00+00:00
5    2023-01-14 05:35:00+00:00
6    2023-01-14 07:15:00+00:00
Name: departure_time_local, dtype: datetime64[ns, UTC]

## 3.Load

### load the flight info to Local MySQL

In [19]:
flight_c_df.to_sql('flight',con=con,if_exists='append',index=False)

82